In [ ]:
import numpy as np
from pychunkedgraph.graph import ChunkedGraph, ChunkedGraphMeta
from pychunkedgraph.graph.meta import ChunkedGraphMeta, GraphConfig, DataSource

ds = DataSource(
  EDGES= "gs://chunkedgraph/minnie65_0/edges",
  COMPONENTS= "gs://chunkedgraph/minnie65_0/components",
  WATERSHED= "gs://microns-seunglab/minnie65/ws_minnie65_0"
)
gc = GraphConfig(
  CHUNK_SIZE= [256, 256, 512]
)
cg = ChunkedGraph(graph_id="akhilesh-minnie65-1")

In [ ]:
data = {
  "sources": [
    [
      "91356497812394262",
      815458.9375,
      884707.6875,
      859720
    ]
  ],
  "sinks": [
    [
      "91356497812401228",
      816143.625,
      884547.3125,
      859720
    ]
  ]
}

In [ ]:
from collections import defaultdict
data_dict = {}
for k in ["sources", "sinks"]:
    data_dict[k] = defaultdict(list)
    for node in data[k]:
        node_id = node[0]
        x, y, z = node[1:]
        coordinate = np.array([x, y, z]) / cg.meta._ws_cv.resolution

        atomic_id = cg.get_atomic_id_from_coord(
            coordinate[0],
            coordinate[1],
            coordinate[2],
            parent_id=np.uint64(node_id),
        )

        if atomic_id is None:
            raise ValueError("aha")

        data_dict[k]["id"].append(atomic_id)
        data_dict[k]["coord"].append(coordinate)
data_dict

In [ ]:
from pychunkedgraph.graph.operation import MulticutOperation

op = MulticutOperation(
    cg,
    user_id="test",
    source_ids=data_dict["sources"]["id"],
    sink_ids=data_dict["sinks"]["id"],
    source_coords=data_dict["sources"]["coord"],
    sink_coords=data_dict["sinks"]["coord"],    
    bbox_offset=(240,240,24)
)

op._apply(operation_id="",timestamp=None)